# **AI OPTIMIZER**





## **PHASE I**
   
   **Natural Language Processing**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

# For text processing
import re
from textblob import TextBlob

# For deep learning
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")

✅ All libraries imported successfully!
🔥 CUDA Available: False


**Using GoEmotions dataset for fine modeling and prototype**



In [4]:
# Download GoEmotions dataset
from datasets import load_dataset

dataset = load_dataset("go_emotions", "simplified")

print("loaidng Dataset")
print(f"Training samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"Test samples: {len(dataset['test'])}")

# Convert to pandas for easy viewing
train_df = pd.DataFrame(dataset['train'])
print("\nFirst few samples:")
print(train_df.head())

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

loaidng Dataset
Training samples: 43410
Validation samples: 5426
Test samples: 5427

First few samples:
                                                text labels       id
0  My favourite food is anything I didn't have to...   [27]  eebbqej
1  Now if he does off himself, everyone will thin...   [27]  ed00q6i
2                     WHY THE FUCK IS BAYLESS ISOING    [2]  eezlygj
3                        To make her feel threatened   [14]  ed7ypvh
4                             Dirty Southern Wankers    [3]  ed0bdzj


In [5]:
# emotion distribution in training dataset
emotion_names = dataset['train'].features['labels'].feature.names
print("\n🎭 Available emotions:")
for i, emotion in enumerate(emotion_names):
    print(f"{i}: {emotion}")

# Count emotions
def count_emotions(dataset_split):
    emotion_counts = {emotion: 0 for emotion in emotion_names}

    for example in dataset_split:
        for label_id in example['labels']:
            emotion_counts[emotion_names[label_id]] += 1

    return emotion_counts

emotion_counts = count_emotions(dataset['train'])


🎭 Available emotions:
0: admiration
1: amusement
2: anger
3: annoyance
4: approval
5: caring
6: confusion
7: curiosity
8: desire
9: disappointment
10: disapproval
11: disgust
12: embarrassment
13: excitement
14: fear
15: gratitude
16: grief
17: joy
18: love
19: nervousness
20: optimism
21: pride
22: realization
23: relief
24: remorse
25: sadness
26: surprise
27: neutral


In [6]:
# Visualize
plt.figure(figsize=(15, 6))
emotions = list(emotion_counts.keys())
counts = list(emotion_counts.values())

# 3d visualization for emotion distribution
emotion_df = pd.DataFrame({'Emotion': emotions, 'Count': counts})

fig = px.bar(emotion_df, x='Emotion', y='Count', color='Emotion', title='<b>Emotion Distribution in GoEmotions Dataset')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

print(f"\n📈 Most common emotion: {max(emotion_counts, key=emotion_counts.get)}")
print(f"📉 Least common emotion: {min(emotion_counts, key=emotion_counts.get)}")


📈 Most common emotion: neutral
📉 Least common emotion: grief


<Figure size 1500x600 with 0 Axes>

we are going to used pre-trained model called **"SamLowe/roberta-base-go_emotions"** from transformers libraray using hugging face

In [7]:
#  Using a model fine-tuned on GoEmotions
model_name = "SamLowe/roberta-base-go_emotions"

emotion_classifier = pipeline(
    "text-classification",
    model=model_name,
    top_k=None,  # Return all emotions with scores
    device=0 if torch.cuda.is_available() else -1
)

print("✅ Model loaded successfully!")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cpu


✅ Model loaded successfully!


Testing with Sample messages to detect model performance

In [8]:
# Testing with sample employee messages
test_messages = [
    "I'm so excited about this new project! Can't wait to get started!",
    "I'm feeling really overwhelmed with all these deadlines.",
    "I don't know if I can handle any more work right now.",
    "Great job everyone! We hit our target!",
    "I'm worried about the upcoming presentation."
]

print("🧪 Testing Emotion Detection:\n")
print("="*80)

for msg in test_messages:
    results = emotion_classifier(msg)[0]

    # Sort by score
    results_sorted = sorted(results, key=lambda x: x['score'], reverse=True)

    print(f"\n📝 Message: {msg}")
    print(f"🎭 Top 3 Emotions:")
    for i, result in enumerate(results_sorted[:3], 1):
        print(f"   {i}. {result['label']}: {result['score']:.3f} ({result['score']*100:.1f}%)")
    print("-"*80)

🧪 Testing Emotion Detection:


📝 Message: I'm so excited about this new project! Can't wait to get started!
🎭 Top 3 Emotions:
   1. excitement: 0.818 (81.8%)
   2. curiosity: 0.053 (5.3%)
   3. joy: 0.040 (4.0%)
--------------------------------------------------------------------------------

📝 Message: I'm feeling really overwhelmed with all these deadlines.
🎭 Top 3 Emotions:
   1. annoyance: 0.072 (7.2%)
   2. excitement: 0.066 (6.6%)
   3. disappointment: 0.061 (6.1%)
--------------------------------------------------------------------------------

📝 Message: I don't know if I can handle any more work right now.
🎭 Top 3 Emotions:
   1. confusion: 0.842 (84.2%)
   2. disapproval: 0.123 (12.3%)
   3. disappointment: 0.113 (11.3%)
--------------------------------------------------------------------------------

📝 Message: Great job everyone! We hit our target!
🎭 Top 3 Emotions:
   1. admiration: 0.944 (94.4%)
   2. approval: 0.042 (4.2%)
   3. gratitude: 0.020 (2.0%)
------------------

In [9]:
# Map detailed emotions to workplace-relevant categories
def categorize_workplace_emotion(emotion_results):

    # emotion mappings
    positive_emotions = ['admiration', 'amusement', 'approval', 'caring', 'desire',
                        'excitement', 'gratitude', 'joy', 'love', 'optimism', 'pride', 'relief']

    negative_emotions = ['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust',
                        'embarrassment', 'fear', 'grief', 'nervousness', 'remorse', 'sadness']

    stress_indicators = ['fear', 'nervousness', 'sadness', 'grief', 'disappointment']

    neutral_emotions = ['confusion', 'curiosity', 'realization', 'surprise', 'neutral']

    #
    positive_score = 0
    for r in emotion_results:
        if r['label'] in positive_emotions:
            positive_score += r['score']

    negative_score = 0
    for r in emotion_results:
        if r['label'] in negative_emotions:
            negative_score += r['score']

    stress_score = 0
    for r in emotion_results:
        if r['label'] in stress_indicators:
            stress_score += r['score']

    neutral_score = 0
    for r in emotion_results:
        if r['label'] in neutral_emotions:
            neutral_score += r['score']

    # Determine primary category
    scores = {
        'Positive': positive_score,
        'Negative': negative_score,
        'Neutral': neutral_score
    }

    primary_category = max(scores, key=scores.get)

    return {
        'category': primary_category,
        'positive_score': positive_score,
        'negative_score': negative_score,
        'stress_score': stress_score,
        'neutral_score': neutral_score,
        'needs_attention': stress_score > 0.5 or negative_score > 0.6
    }

# Test the categorization
print("🏢 Workplace Emotion Categorization:\n")
print("="*80)

for message in test_messages:
    results = emotion_classifier(message)[0]
    categorization = categorize_workplace_emotion(results)

    print(f"\n📝 Message: {message}")
    print(f"🎯 Category: {categorization['category']}")
    print(f"📊 Scores:")
    print(f"   Positive: {categorization['positive_score']:.3f}")
    print(f"   Negative: {categorization['negative_score']:.3f}")
    print(f"   Stress: {categorization['stress_score']:.3f}")

    if categorization['needs_attention']:
        print(f"⚠️  ALERT: Employee may need support!")

    print("-"*80)

🏢 Workplace Emotion Categorization:


📝 Message: I'm so excited about this new project! Can't wait to get started!
🎯 Category: Positive
📊 Scores:
   Positive: 0.948
   Negative: 0.028
   Stress: 0.013
--------------------------------------------------------------------------------

📝 Message: I'm feeling really overwhelmed with all these deadlines.
🎯 Category: Negative
📊 Scores:
   Positive: 0.148
   Negative: 0.265
   Stress: 0.168
--------------------------------------------------------------------------------

📝 Message: I don't know if I can handle any more work right now.
🎯 Category: Neutral
📊 Scores:
   Positive: 0.097
   Negative: 0.394
   Stress: 0.208
--------------------------------------------------------------------------------

📝 Message: Great job everyone! We hit our target!
🎯 Category: Positive
📊 Scores:
   Positive: 1.049
   Negative: 0.011
   Stress: 0.004
--------------------------------------------------------------------------------

📝 Message: I'm worried about th

In [10]:
# Task recommendation based on emotions
def recommend_task(emotion_category, stress_score, positive_score):
    """
    Recommending tasks based on detected emotion
    """

    recommendations = {
        'Positive': {
            'high_energy': [
                "🎨 Creative brainstorming session",
                "🎯 Tackle complex problem-solving tasks",
                "🗣️ Lead team meeting or presentation",
                "📈 Strategic planning activities",
                "🤝 Client-facing interactions"
            ],
            'moderate': [
                "📝 Write detailed documentation",
                "🔍 Code review and optimization",
                "🎓 Mentor junior team members",
                "💡 Prototype new features",
                "📊 Data analysis and insights"
            ]
        },
        'Neutral': [
            "📋 Administrative tasks and emails",
            "🗂️ Organize and document work",
            "📖 Review and proofread content",
            "🔄 Routine maintenance tasks",
            "📚 Learning and training modules"
        ],
        'Negative': {
            'low_stress': [
                "🤝 Collaborative team tasks",
                "✅ Well-defined, structured activities",
                "🎯 Small, achievable milestones",
                "🔄 Switch to different project area",
                "👥 Pair programming with support"
            ],
            'high_stress': [
                "⏸️ Take a scheduled break",
                "🧘 Mindfulness or relaxation exercise",
                "💬 Have a check-in conversation",
                "📝 Light administrative work only",
                "🌟 Delegate complex tasks temporarily"
            ]
        }
    }

    if emotion_category == 'Positive':
        if positive_score > 0.7:
            tasks = recommendations['Positive']['high_energy']
            energy_level = "High Energy"
        else:
            tasks = recommendations['Positive']['moderate']
            energy_level = "Moderate Energy"
    elif emotion_category == 'Neutral':
        tasks = recommendations['Neutral']
        energy_level = "Balanced"
    else:  # Negative
        if stress_score > 0.5:
            tasks = recommendations['Negative']['high_stress']
            energy_level = "High Stress - Needs Support"
        else:
            tasks = recommendations['Negative']['low_stress']
            energy_level = "Low Mood"

    return {
        'energy_level': energy_level,
        'recommended_tasks': tasks
    }

# Test recommendations
print("💼 Task Recommendations Based on Emotions:\n")
print("="*80)

for message in test_messages:
    results = emotion_classifier(message)[0]
    categorization = categorize_workplace_emotion(results)
    recommendations = recommend_task(
        categorization['category'],
        categorization['stress_score'],
        categorization['positive_score']
    )

    print(f"\n📝 Message: {message}")
    print(f"🎯 Energy Level: {recommendations['energy_level']}")
    print(f"✅ Recommended Tasks:")
    for i, task in enumerate(recommendations['recommended_tasks'], 1):
        print(f"   {i}. {task}")
    print("-"*80)

💼 Task Recommendations Based on Emotions:


📝 Message: I'm so excited about this new project! Can't wait to get started!
🎯 Energy Level: High Energy
✅ Recommended Tasks:
   1. 🎨 Creative brainstorming session
   2. 🎯 Tackle complex problem-solving tasks
   3. 🗣️ Lead team meeting or presentation
   4. 📈 Strategic planning activities
   5. 🤝 Client-facing interactions
--------------------------------------------------------------------------------

📝 Message: I'm feeling really overwhelmed with all these deadlines.
🎯 Energy Level: Low Mood
✅ Recommended Tasks:
   1. 🤝 Collaborative team tasks
   2. ✅ Well-defined, structured activities
   3. 🎯 Small, achievable milestones
   4. 🔄 Switch to different project area
   5. 👥 Pair programming with support
--------------------------------------------------------------------------------

📝 Message: I don't know if I can handle any more work right now.
🎯 Energy Level: Balanced
✅ Recommended Tasks:
   1. 📋 Administrative tasks and emails
   2. 🗂️

In [11]:
# Create synthetic employee data for demonstration
np.random.seed(42)

def generate_employee_messages(num_days=7):
    """
    Simulate employee messages over time
    """

    message_templates = {
        'positive': [
            "Great progress on the project today!",
            "Really excited about the new features we're building",
            "Had a productive meeting with the team",
            "Feeling motivated to tackle these challenges",
            "Love working with this team!"
        ],
        'neutral': [
            "Working on the documentation today",
            "Reviewing code and making updates",
            "Normal day at work, handling routine tasks",
            "Going through emails and planning",
            "Steady progress on assigned tasks"
        ],
        'stressed': [
            "Feeling overwhelmed with the workload",
            "Too many deadlines approaching",
            "Struggling to keep up with everything",
            "This is getting really stressful",
            "Not sure I can handle all of this"
        ],
        'frustrated': [
            "Another meeting that should have been an email",
            "Dealing with the same issues again",
            "This is taking way longer than expected",
            "Really frustrated with these blockers",
            "Why is this so difficult?"
        ]
    }

    employees = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']
    data = []

    start_date = datetime.now() - timedelta(days=num_days-1)

    for day in range(num_days):
        current_date = start_date + timedelta(days=day)

        for employee in employees:
            # Simulate different patterns for different employees
            if employee == 'Charlie':  # Charlie is experiencing burnout
                if day < 3:
                    mood = np.random.choice(['neutral', 'positive'], p=[0.7, 0.3])
                else:
                    mood = np.random.choice(['stressed', 'frustrated'], p=[0.6, 0.4])
            elif employee == 'Diana':  # Diana is consistently positive
                mood = np.random.choice(['positive', 'neutral'], p=[0.7, 0.3])
            else:  # Others have mixed moods
                mood = np.random.choice(['positive', 'neutral', 'stressed', 'frustrated'],
                                       p=[0.4, 0.3, 0.2, 0.1])

            message = np.random.choice(message_templates[mood])

            data.append({
                'date': current_date.strftime('%Y-%m-%d'),
                'employee': employee,
                'message': message,
                'time': f"{np.random.randint(9, 18)}:{np.random.randint(0, 59):02d}"
            })

    return pd.DataFrame(data)

# Generate employee data
employee_df = generate_employee_messages(num_days=7)
print("👥 Generated Employee Messages:\n")
print(employee_df.head(10))
print(f"\nTotal messages: {len(employee_df)}")

👥 Generated Employee Messages:

         date employee                                 message   time
0  2025-10-22    Alice            Love working with this team!  16:20
1  2025-10-22      Bob  Had a productive meeting with the team  15:10
2  2025-10-22  Charlie       Steady progress on assigned tasks  12:39
3  2025-10-22    Diana       Steady progress on assigned tasks  10:23
4  2025-10-22      Eve          Too many deadlines approaching  13:32
5  2025-10-23    Alice            Love working with this team!  17:48
6  2025-10-23      Bob       Reviewing code and making updates  11:43
7  2025-10-23  Charlie      Working on the documentation today  11:36
8  2025-10-23    Diana            Love working with this team!  17:38
9  2025-10-23      Eve    Great progress on the project today!  10:57

Total messages: 35


In [12]:
# Process employee messages to get emotion analysis results
print("⏳ Analyzing employee messages...")

results_list = []
for index, row in employee_df.iterrows():
    message = row['message']
    employee = row['employee']
    date = row['date']

    # Analyze emotion from text
    emotion_results = emotion_classifier(message)[0]

    # Categorize for workplace context
    categorization = categorize_workplace_emotion(emotion_results)

    # Get dominant emotion from detailed results
    dominant_emotion = max(emotion_results, key=lambda x: x['score'])['label']

    results_list.append({
        'date': date,
        'employee': employee,
        'message': message,
        'dominant_emotion': dominant_emotion,
        'category': categorization['category'],
        'positive_score': categorization['positive_score'],
        'negative_score': categorization['negative_score'],
        'stress_score': categorization['stress_score'],
        'needs_attention': categorization['needs_attention']
    })

results_df = pd.DataFrame(results_list)

print("✅ Analysis complete! Results stored in results_df")
display(results_df.head())

⏳ Analyzing employee messages...
✅ Analysis complete! Results stored in results_df


date employee                                 message  \
0  2025-10-22    Alice            Love working with this team!   
1  2025-10-22      Bob  Had a productive meeting with the team   
2  2025-10-22  Charlie       Steady progress on assigned tasks   
3  2025-10-22    Diana       Steady progress on assigned tasks   
4  2025-10-22      Eve          Too many deadlines approaching   

  dominant_emotion  category  positive_score  negative_score  stress_score  \
0             love  Positive        1.115025        0.024005      0.009101   
1              joy  Positive        0.922268        0.022827      0.008372   
2          neutral   Neutral        0.098095        0.022647      0.014848   
3          neutral   Neutral        0.098095        0.022647      0.014848   
4          neutral   Neutral        0.079648        0.083112      0.044495   

   needs_attention  
0            False  
1            False  
2            False  
3            False  
4            False

In [13]:
# Identify employees needing attention
print("⚠️ STRESS ALERT SYSTEM\n")
print("="*80)

# Define alert criteria
STRESS_THRESHOLD = 0.4  # Average stress score threshold
CONSECUTIVE_DAYS = 2     # Number of consecutive days to trigger alert

alerts = []

for employee in results_df['employee'].unique():
    employee_data = results_df[results_df['employee'] == employee].sort_values('date')

    # Calculate daily average stress
    daily_stress = employee_data.groupby('date')['stress_score'].mean()

    # Check for prolonged stress
    consecutive_high_stress = 0
    for stress in daily_stress:
        if stress > STRESS_THRESHOLD:
            consecutive_high_stress += 1
            if consecutive_high_stress >= CONSECUTIVE_DAYS:
                alerts.append({
                    'employee': employee,
                    'avg_stress': daily_stress.mean(),
                    'days_stressed': consecutive_high_stress,
                    'severity': 'HIGH' if daily_stress.mean() > 0.6 else 'MEDIUM'
                })
                break
        else:
            consecutive_high_stress = 0

if alerts:
    print("🚨 EMPLOYEES REQUIRING IMMEDIATE ATTENTION:\n")
    for alert in alerts:
        print(f"👤 Employee: {alert['employee']}")
        print(f"📊 Average Stress Level: {alert['avg_stress']:.3f}")
        print(f"📅 Days with High Stress: {alert['days_stressed']}")
        print(f"⚠️  Severity: {alert['severity']}")
        print(f"💡 Recommendation: Schedule check-in, offer support resources")
        print("-"*80)
else:
    print("✅ No employees currently require immediate attention")

print(f"\n📈 Overall Team Health:")
print(f"   Average Positive Score: {results_df['positive_score'].mean():.3f}")
print(f"   Average Negative Score: {results_df['negative_score'].mean():.3f}")
print(f"   Average Stress Score: {results_df['stress_score'].mean():.3f}")

⚠️ STRESS ALERT SYSTEM

🚨 EMPLOYEES REQUIRING IMMEDIATE ATTENTION:

👤 Employee: Charlie
📊 Average Stress Level: 0.258
📅 Days with High Stress: 2
⚠️  Severity: MEDIUM
💡 Recommendation: Schedule check-in, offer support resources
--------------------------------------------------------------------------------

📈 Overall Team Health:
   Average Positive Score: 0.480
   Average Negative Score: 0.134
   Average Stress Score: 0.104


In [14]:
results_df.to_csv('employee_emotion_analysis.csv', index=False)
print("💾 Results saved to 'employee_emotion_analysis.csv'")


💾 Results saved to 'employee_emotion_analysis.csv'


In [15]:
# Install DeepFace and required libraries
!pip install deepface
!pip install opencv-python-headless
!pip install tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00


In [16]:
import cv2

In [17]:
#import warnings
#warnings.filterwarnings("ignore")

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from deepface import DeepFace


25-10-28 11:34:43 - Directory /root/.deepface has been created
25-10-28 11:34:43 - Directory /root/.deepface/weights has been created


In [18]:
img = cv2.imread(r'/content/girl.jpg')

In [19]:
import matplotlib.pyplot as plt

In [20]:
if img is not None:
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
else:
    print("Error: Image not loaded. Check the file path and ensure the image exists.")

Error: Image not loaded. Check the file path and ensure the image exists.


In [21]:
predictions = DeepFace.analyze(img)

ValueError: img must be numpy array or str but it is <class 'NoneType'>

In [ ]:
predictions

In [ ]:
predictions = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)

# Access the dominant emotion correctly:
dominant_emotion = predictions[0]['dominant_emotion']
print(dominant_emotion)

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(gray,1.1,4)
for(x, y, w, h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 5)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX

cv2.putText(img,
            predictions[0]['dominant_emotion'],
            (0,50),
            font,3,
            (0, 0, 255),
            2,
            cv2.LINE_4);


In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
img = cv2.imread(r'/surprise.jpg')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
prediction = DeepFace.analyze(img)

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(gray,1.1,4)
for(x, y, w, h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 3)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX

cv2.putText(img,
            prediction[0]['dominant_emotion'],
            (0,50),
            font,2,
            (0, 0, 255),
            2,
            cv2.LINE_4);

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
import glob
import os
path = '/content/'

images = glob.glob(os.path.join(path, '*.jpg')) + \
         glob.glob(os.path.join(path, '*.png'))

sample_images_paths = []


for img_path in images:
    sample_images_paths.append(("unknown_emotion", img_path)) # Assign a placeholder emotion

print(f"✅ Found {len(sample_images_paths)} sample images\n")


In [ ]:
def analyze_face_emotion(image_path, show_image=False):

    try:
        # Read and display image
        img = cv2.imread(image_path)

        if img is None:
            return {
                'success': False,
                'error': f"Failed to load image from {image_path}. The file might be corrupted or not accessible."
            }

        if show_image:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            plt.figure(figsize=(4, 4))
            plt.imshow(img_rgb)
            plt.axis('off')
            plt.title("Analyzing...")
            # plt.show()


        # Analyze with DeepFace
        result = DeepFace.analyze(
            img_path=image_path,
            actions=['emotion', 'age', 'gender'],
            enforce_detection=False
        )

        # Handle multiple faces
        if isinstance(result, list):
            result = result[0]

        # Extract emotion data
        emotions = result['emotion']
        dominant_emotion = result['dominant_emotion']

        # Categorize for workplace context
        positive_emotions = ['happy', 'surprise']
        negative_emotions = ['angry', 'disgust', 'fear', 'sad']

        positive_score = sum(emotions[e] for e in positive_emotions if e in emotions) / 100
        negative_score = sum(emotions[e] for e in negative_emotions if e in emotions) / 100
        neutral_score = emotions.get('neutral', 0) / 100

        # Calculate stress indicators
        stress_score = (emotions.get('fear', 0) + emotions.get('sad', 0)) / 200

        # Determine category
        if positive_score > negative_score and positive_score > neutral_score:
            category = 'Positive'
        elif negative_score > positive_score and negative_score > neutral_score:
            category = 'Negative'
        else:
            category = 'Neutral'

        return {
            'success': True,
            'emotions': emotions,
            'dominant_emotion': dominant_emotion,
            'category': category,
            'positive_score': positive_score,
            'negative_score': negative_score,
            'neutral_score': neutral_score,
            'stress_score': stress_score,
            'needs_attention': stress_score > 0.3 or negative_score > 0.5,
            'age': result.get('age'),
            'gender': result.get('dominant_gender')
        }

    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

print("🧪 Testing facial emotion analysis function:\n")

if 'sample_images_paths' in locals() and sample_images_paths:
    test_emotion, test_img_path = sample_images_paths[0]
    # Modified to show image
    result = analyze_face_emotion(test_img_path, show_image=True)

    if result['success']:
        print("✅ Analysis successful!")
        print(f"\n📊 Results:")
        print(f"   Dominant Emotion: {result['dominant_emotion']}")
        print(f"   Category: {result['category']}")
        print(f"   Positive Score: {result['positive_score']:.3f}")
        print(f"   Negative Score: {result['negative_score']:.3f}")
        print(f"   Stress Score: {result['stress_score']:.3f}")
        print(f"   Needs Attention: {result['needs_attention']}")
    else:
        print(f"❌ Error: {result['error']}")

In [ ]:
!pip install streamlit

In [ ]:
# Enable webcam access in Colab
from IPython.display import display, Javascript, Image as IPImage
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io

def take_photo(filename='photo.jpg', quality=0.8):
    """
    Take a photo using webcam in Colab
    """
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '📸 Capture Photo';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))

    # Decode and save
    binary = b64decode(data.split(',')[1])

    with open(filename, 'wb') as f:
        f.write(binary)

    return filename

print("📷 Webcam Emotion Detection Setup")
print("=" * 60)
print("Click the button above to capture a photo from your webcam")
print("The system will analyze your facial emotion in real-time!")
print("=" * 60)

# real time face
photo_path = take_photo('my_emotion.jpg')
result = analyze_face_emotion(photo_path, show_image=True)

if result['success']:
    print("✅ Analysis successful!")
    print(f"\n📊 Results:")
    print(f"   Dominant Emotion: {result['dominant_emotion']}")
    print(f"   Category: {result['category']}")
    print(f"   Positive Score: {result['positive_score']:.3f}")
    print(f"   Negative Score: {result['negative_score']:.3f}")
    print(f"   Stress Score: {result['stress_score']:.3f}")
    print(f"   Needs Attention: {result['needs_attention']}")
if result['needs_attention']:
        print("⚠️ Alert: You might benefit from a break or support!")
else:
    print(f"❌ Error: {result['error']}")


In [ ]:
import streamlit as st
from PIL import Image
import io
import numpy as np
import cv2
import base64

# Assuming analyze_face_emotion and recommend_task functions are defined in previous cells
# from your_emotion_analysis_module import analyze_face_emotion, recommend_task # In a real app, you'd import

st.title("🧠 AI Employee Mood Optimizer")

st.write("Analyze your mood and get personalized task recommendations.")

# Option to use webcam or upload image
option = st.radio("Choose input method:", ('Upload Image', 'Use Webcam'))

if option == 'Upload Image':
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption='Uploaded Image.', use_column_width=True)

        # Convert PIL Image to OpenCV format
        img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

        st.write("")
        st.write("Analyzing...")

        # Analyze emotion
        result = analyze_face_emotion(img_cv, show_image=False) # Pass OpenCV image directly if analyze_face_emotion can handle it

        if result['success']:
            st.success("Analysis Successful!")
            st.write(f"**Dominant Emotion:** {result['dominant_emotion']}")
            st.write(f"**Mood Category:** {result['category']}")
            st.write(f"**Stress Level:** {result['stress_score']:.2f}")

            # Get task recommendations
            recommendations = recommend_task(
                result['category'],
                result['stress_score'],
                result['positive_score']
            )

            st.write(f"**Energy Level:** {recommendations['energy_level']}")
            st.write("**Recommended Tasks:**")
            for task in recommendations['recommended_tasks']:
                st.write(f"- {task}")

            if result['needs_attention']:
                st.warning("⚠️ Alert: You might benefit from a break or support!")

        else:
            st.error(f"Error analyzing image: {result['error']}")

elif option == 'Use Webcam':
    st.write("Webcam functionality will be implemented here.")
    # This part will require more complex Streamlit components or custom JavaScript to capture webcam feed directly.
    # For a simple prototype, you could use the existing Colab webcam code to capture a single frame.